# Ref-A
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 1.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/Ref-A-1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/Ref-A-1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+245.gd0d9c92.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'Ref-A-1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.2
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/Ref-A-1
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json


In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-22_1700', endtime='2022-01-22_1800')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~0 skyfiles. 
Created 0 skyfiles in /skies/


{}

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~0 .rad files for gendaylit 1-axis workflow (this takes a minute..)
0 Radfiles created in /objects/

Making 0 octfiles in root directory.


In [14]:
#trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

IndexError: list index out of range

In [ ]:
#trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

In [ ]:
demo.calculateResults()

In [ ]:
demo.CompiledResults.set_index('timestamp', inplace=True)
dataframe = demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

In [ ]:
dataframe

In [ ]:
# dataframe.to_csv(Path().resolve().parent.parent / 'Results' / 'Ref-A.csv', index=True)

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'Ref-A.csv')